# Section 3: Secondary Analyses
## Individual Regressions on DLPFC-5 LH 
Here we look for the fraction of subjects that show the same DBSon/DBSoff effects in DLPFC-5 LH at the single-subject level.
### Perform permutations
Please see: /space/sophia/2/users/EMOTE-DBS/afMSIT/scripts/cmdline/subject_permutations.py
### Identify clusters, perform FDR corrections, and assemble results.

In [ ]:
import os
import numpy as np
from collections import defaultdict
from mne.stats import fdr_correction
from pandas import DataFrame, concat
from scipy.ndimage import measurements
from scipy.stats import norm

def largest_cluster(arr, threshold):
    masked = np.abs( arr ) > threshold
    clusters, n_clusters = measurements.label(masked)
    cluster_sums = measurements.sum(arr, clusters, index=np.arange(n_clusters)+1)
    if not len(cluster_sums): return 0
    else: return np.abs(cluster_sums).max()

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Specify parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## File parameters.
subjects = ['BRTU', 'CHDR', 'CRDA', 'JASE', 'M5', 'MEWA', 'S2']
labels = ['dlpfc_5-lh']

space = 'source'
analysis = 'stim'
model_name = 'revised'
freqs = ['theta']
domain = ['timedomain', 'frequency'][1]

## Statistics parameters.
alpha = 0.05
min_cluster = 0.05 # seconds

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Initial preparations.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Define contrasts.
if model_name == 'revised': cols = ['Intercept', 'DBS', 'Interference', 'DBSxInt', 'nsArousal', 'nsValence', 'Trial']

## Define threshold.
threshold = norm.ppf(1 - alpha/2.)

## Read in seeds.
f = '/space/sophia/2/users/EMOTE-DBS/afMSIT/scripts/cmdline/seeds.txt'
with open(f, 'r') as f: seeds = [s.strip() for s in f.readlines()]
    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main Loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#    

df = []
    
for subject in subjects: 
        
    for label in labels:

        for freq in freqs:

            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
            ### Load files. Assemble permutations.
            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
            root_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/afMSIT/%s' %space
            results_dir = os.path.join(root_dir, model_name)
            out_dir = os.path.join(root_dir, 'results')

            ## Load true statistics.
            npz = np.load(os.path.join(results_dir, '%s_%s_%s_%s_%s_obs.npz' %(subject, space, analysis, label, freq)))
            t_scores = npz['t_scores'].squeeze()
            times = npz['times'].squeeze()

            ## Load null statistics.
            for n, seed in enumerate(seeds):
                npz = np.load(os.path.join(results_dir, '%s_%s_%s_%s_%s_%s.npz' %(subject, space, analysis, label, freq, seed)))
                if not n: permuted = npz['t_scores']
                else: permuted = np.concatenate([permuted, npz['t_scores']], axis=0)

            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
            ### Compute cluster statistics.
            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

            ## Get info.
            n_shuffles, n_eff, n_times  = permuted.shape

            ## Iteratively compute clusters.
            results = defaultdict(list)

            for n, con in enumerate(cols):

                ## Find real clusters.
                masked = np.abs( t_scores[n] ) > threshold
                clusters, n_clusters = measurements.label(masked)
                cluster_sums = measurements.sum(t_scores[n], clusters, index=np.arange(n_clusters)+1)

                ## Compute null cluster sums.
                null_sums = np.array([largest_cluster(permuted[m, n, :], threshold) for m in xrange(n_shuffles)])

                ## Compute cluster bounds.
                tmin = np.array([times[clusters==i].min() for i in np.arange(n_clusters)+1])
                tmax = np.array([times[clusters==i].max() for i in np.arange(n_clusters)+1])

                ## Find proportion of clusters that are larger.
                p_values = [((np.abs(cs) < null_sums).sum() + 1.) / (null_sums.shape[0] + 1.) for cs in cluster_sums]

                ## Store results.
                for t1, t2, cs, pval in zip(tmin,tmax,cluster_sums,p_values):
                    results['Subject'] += [subject]
                    results['Freq'] += [freq]
                    results['Label'] += [label]
                    results['Contrast'] += [con]
                    results['Tmin'] += [t1]
                    results['Tmax'] += [t2]
                    results['Score'] += [cs]
                    results['Pval'] += [pval]

            ## Organize results and append.
            results = DataFrame(results)
            results['Tdiff'] = results['Tmax'] - results['Tmin']
            results = results[results['Tdiff']>=min_cluster]
            df.append(results)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Compute cluster statistics.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Merge dataframes.
df = concat(df)

## Organize columns and sort.
cols = ['Subject','Contrast','Label','Freq','Tmin','Tmax','Tdiff','Score','Pval']
df = df[cols].sort_values(['Subject','Contrast','Tmin']).reset_index(drop=True)

## FDR correct within contrasts.
df['FDR'] = 0
for contrast in df.Contrast.unique():
    _, fdr = fdr_correction(df.loc[df.Contrast==contrast,'Pval'], alpha)
    df.loc[df.Contrast==contrast,'FDR'] = fdr
    
## Save.
f = os.path.join(out_dir, 'subjects_%s_%s_%s_results.csv' %(model_name, analysis, domain))
df.to_csv(f, index=False)
        
print 'Done.'

df[(df.Contrast=='DBS')&(df.FDR<0.05)]

## Reaction Time Correlations
### Perform permutations
Please see: /space/sophia/2/users/EMOTE-DBS/afMSIT/scripts/cmdline/afMSIT_permutations.py
### Identify clusters, perform FDR corrections, and assemble results.

In [ ]:
import os
import numpy as np
from pandas import concat, read_csv
from mne.stats import fdr_correction

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O Parameters.
space = 'source'
analysis = 'resp'
labels = ['dacc-lh', 'dacc-rh', 'dmpfc-lh', 'dmpfc-rh', 'dlpfc_1-lh', 'dlpfc_1-rh', 'dlpfc_2-lh', 'dlpfc_2-rh', 
          'dlpfc_3-lh', 'dlpfc_3-rh', 'dlpfc_4-lh', 'dlpfc_4-rh', 'dlpfc_5-lh', 'dlpfc_5-rh', 
          'dlpfc_6-lh', 'dlpfc_6-rh', 'pcc-lh', 'pcc-rh', 'racc-lh', 'racc-rh']
domain = 'timedomain'
freqs = ['15']
model = 'revised'

## Cluster parameters.
min_cluster = 0.05 # secs
alpha = 0.05

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/%s/rt' %space
results_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/%s/results' %space

df = []
for label in labels:
    
    for freq in freqs:
        
        df.append( read_csv(os.path.join(root_dir, 'afMSIT_%s_%s_%s_%s_rt.csv' %(space, analysis, label, freq))) )

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Merge, trim, correct, and save.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
## Merge and trim.
df = concat(df)
df = df[df.Tdiff > min_cluster]

## FDR correct.
df['FDR'] = fdr_correction(df.Pval, alpha=alpha)[-1]

## Save.
f = os.path.join(results_dir, '%s_%s_%s_rt.csv' %(model, analysis, domain))
df.to_csv(f, index=False)
print 'Done.'

## MADRS Correlations
### Permutations for MADRS Correlations
Please see: /space/sophia/2/users/EMOTE-DBS/afMSIT/scripts/cmdline/afMSIT_permutations.py

In [ ]:
import os
import numpy as np
from pandas import DataFrame
np.random.seed(47404)

## Define parameters.
n_subj = 8
n_shuffles = 1000

## Prepare
ix = np.arange(n_subj)
permutations = np.expand_dims(ix[::-1], 0)
    
while True: 
    np.random.shuffle(ix)
    permutations = np.concatenate([permutations, np.expand_dims(ix, 0)], axis=0)
    permutations = DataFrame(permutations).drop_duplicates()
    if permutations.shape[0] < n_shuffles: permutations = permutations.as_matrix()
    else: break
        
## Save.
np.save('/space/sophia/2/users/EMOTE-DBS/afMSIT/scripts/cmdline/madrs_permutations', permutations.as_matrix())

### MADRS Correlations

In [14]:
import os
import numpy as np
from pandas import read_csv
from scipy.stats import spearmanr
from mne.stats import fdr_correction

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

space = 'source'
analysis = 'stim'
domain = 'frequency'
model = 'revised'
contrast = 'DBS'
freq = 'theta'
fdr = 0.05

subjects = np.array(['BRTU', 'CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2'])

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Prepare MADRS Scores.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/%s' %space
info = read_csv(os.path.join(root_dir, 'afMSIT_%s_info.csv' %space))

## Prepare MADRS scores.
scores = read_csv('behavior/afMSIT_demographics.csv', index_col=0)
subjects = scores.index
madrs = scores['MADRS_Now'] - scores['MADRS_Base']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

results = read_csv(os.path.join(root_dir, 'results', '%s_%s_%s_results.csv' %(model,analysis,domain)))
results = results[(results.Contrast==contrast)&(results.FDR<fdr)&(results.Freq==freq)].reset_index(drop=True)
results['MADRS_Corr_Rval'] = 0 
results['MADRS_Corr_Pval'] = 0 

for n in range(len(results)):
    
    label, freq, tmin, tmax = results.loc[n, ['Label','Freq','Tmin','Tmax']]
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load and prepare data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    npz = np.load(os.path.join(root_dir, 'afMSIT_%s_%s_%s_%s.npz' %(space, analysis, label, freq)))
    data = npz['data']
    times = npz['times']

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Compute condition differences.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    delta = np.zeros(len(subjects))
    mask = (times >= tmin) & (times <= tmax)    
    
    for m, subject in enumerate(subjects):
        i, = np.where((info['Subject']==subject)&(info[contrast]==0))
        j, = np.where((info['Subject']==subject)&(info[contrast]==1))
        delta[m] += (data[j][:,mask].mean(axis=0) - data[i][:,mask].mean(axis=0)).mean()
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Perform correlations.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Compute true correlations.
    r, p = spearmanr(madrs, delta)
    if label == 'dlpfc_5-lh': test = delta.copy()
    results.loc[n, 'MADRS_Corr_Rval'] = r
    results.loc[n, 'MADRS_Corr_Pval'] = p

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Save.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Save.
_, results['MADRS_Corr_Pval'] = fdr_correction(results['MADRS_Corr_Pval'])

f = os.path.join(root_dir, 'results', '%s_%s_%s_madrs.csv' %(model, analysis, domain))
# results.to_csv(f, index=False)
    
print 'Done.'

Done.


In [18]:
results[results.MADRS_Corr_Pval < 0.1]

,Contrast,Label,Freq,Tmin,Tmax,Tdiff,Score,Pval,FDR,MADRS_Corr_Rval,MADRS_Corr_Pval
1,DBS,dlpfc_4-lh,theta,0.340000,0.581379,0.241379,75.537991,0.004995,0.041387,0.805118,0.059239
2,DBS,dlpfc_5-lh,theta,0.378621,0.639310,0.260690,76.045844,0.004995,0.041387,0.853913,0.055722
4,DBS,dlpfc_5-lh,theta,0.658621,1.170345,0.511724,184.423364,0.000999,0.019314,0.780720,0.059239


## DLPFC_5-LH Findings
### Special MADRS Correlation for DLPFC_5-LH (Stimulus-Locked)

In [ ]:
import os
import numpy as np
import pylab as plt
from pandas import read_csv
from scipy.stats import spearmanr
from mne.stats import fdr_correction
np.random.seed(47404)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
space = 'source'
analysis = 'stim'
domain = 'frequency'
model = 'revised'
contrast = 'DBS'
fdr = 0.05

## Subject parameters.
subjects = np.array(['BRTU', 'CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2'])

## Bootstrapping parameters.
n_straps = 1000

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

fig = plt.figure(figsize=(12,12))
true_scores = np.zeros((2,2))
bootstraps = np.zeros((2,n_straps))

## Looping over including and excluding S2.
for n in np.arange(2): 

    if n: subjects = np.delete(subjects, -1)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Prepare MADRS Scores.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/%s' %space
    info = read_csv(os.path.join(root_dir, 'afMSIT_%s_info.csv' %space))

    ## Prepare MADRS scores.
    ratings = read_csv('/space/sophia/2/users/EMOTE-DBS/afMSIT/behavior/Subject_Rating_Scales.csv')
    ratings = ratings.set_index('Subject')
    madrs = ratings.loc[subjects, 'MADRS_Now'] - ratings.loc[subjects, 'MADRS_Base']

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load and prepare data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Load cluster results.
    results = read_csv(os.path.join(root_dir, 'results', '%s_%s_%s_results.csv' %(model,analysis,domain)))
    results = results[(results.Contrast==contrast)&(results.FDR<fdr)].reset_index(drop=True)
    results = results[results.Label == 'dlpfc_5-lh'] # Limit to the theta clusters.

    ## Load time series data.
    npz = np.load(os.path.join(root_dir, 'afMSIT_%s_%s_%s_%s.npz' %(space, analysis, 'dlpfc_5-lh', 'theta')))
    data = npz['data']
    times = npz['times']

    ## Compute condition differences.
    delta = np.zeros(subjects.shape[0])
    mask = (times >= results.Tmin.min()) & (times <= results.Tmax.min())    

    for m, subject in enumerate(subjects):
        i, = np.where((info['Subject']==subject)&(info[contrast]==0))
        j, = np.where((info['Subject']==subject)&(info[contrast]==1))
        delta[m] += (data[j][:,mask].mean(axis=0) - data[i][:,mask].mean(axis=0)).mean()

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Perform correlations.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Compute true correlations.
    true_scores[n] = spearmanr(madrs, delta)
    
    ## Perform bootstrapping
    n_subj = subjects.shape[0]
    for m in np.arange(n_straps):
        
        ix = np.random.choice(np.arange(n_subj), n_subj, replace=True)
        bootstraps[n,m], _ = spearmanr(madrs[ix], delta[ix])
        
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Plot Scatterplot.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ax = plt.subplot2grid((2,2),(0,n))
    if not n: color = '#2b8cbe'
    else: color = '#31a354'

    ## Iteratively add points.
    for x,y,subj in zip(madrs,delta,madrs.index):

        ax.scatter(x,y,s=80,color=color)
        ax.text(x+0.5,y+0.05,subj)

    ## Add trend line.
    x1, x2 = ax.get_xlim()
    y1, y2 = ax.get_ylim()

    z = np.polyfit(madrs, delta, 1)
    p = np.poly1d(z)
    ax.plot(np.linspace(x1,x2,1e3), p(np.linspace(x1,x2,1e3)), linewidth=2, color='k', alpha=0.8)
    ax.set_xlim(x1,x2)
    ax.set_ylim(y1,y2)

    ## Add flourishes.
    ax.set_xlabel('MADRS [Post - Pre]', fontsize=18)
    ax.set_ylabel('Theta Power [On - Off]', fontsize=18)
    if not n: ax.set_title('DLPFC-5 LH (S2 included)', fontsize=24)
    else: ax.set_title('DLPFC-5 LH (S2 discarded)', fontsize=24)
    ax.text(x1+0.5, y2*0.99, 'r = %0.3f\np = %0.3f' %(true_scores[n,0],true_scores[n,1]), 
            fontsize=18, va='top')

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Plot Histogram.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ax = plt.subplot2grid((2,2),(1,n))

    ## Plot histogram.
    x = bootstraps[n]
    x = x[~np.isnan(x)]
    ax.hist(x, bins=40, color=color)

    ## Add true score.
    y1, y2 = ax.get_ylim()
    ax.vlines(true_scores[n,0], y1, y2, linestyle='--', linewidth=2)
    ax.text(true_scores[n,0], y2*0.98, 'r = %0.3f' %true_scores[n,0], 
            ha='right', va='top', rotation='vertical', weight='heavy')
    
    ## Add flourishes.
    ax.set_xlabel('Spearman R', fontsize=18) 
    ax.set_ylabel('Frequency', fontsize=18)
    if not n: ax.set_title('%s Bootstraps (S2 included)' %n_straps, fontsize=20)
    else: ax.set_title('%s Bootstraps (S2 discarded)' %n_straps, fontsize=20)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Save.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#        

plt.tight_layout()
plt.show()

### Special Hypomania Correlation for DLPFC_5-LH (Stimulus-Locked)

In [ ]:
import os
import numpy as np
import pylab as plt
from pandas import read_csv
from scipy.stats import spearmanr
from mne.stats import fdr_correction
np.random.seed(47404)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
space = 'source'
analysis = 'stim'
domain = 'frequency'
model = 'revised'
contrast = 'DBS'
fdr = 0.05

## Subject parameters.
subjects = np.array(['BRTU', 'CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA'])

## Bootstrapping parameters.
n_straps = 1000

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Prepare MADRS Scores.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/%s' %space
info = read_csv(os.path.join(root_dir, 'afMSIT_%s_info.csv' %space))

## Prepare MADRS scores.
ratings = read_csv('/space/sophia/2/users/EMOTE-DBS/afMSIT/behavior/Subject_Characteristics.csv')
ratings = ratings.set_index('Name')
mania = ratings.loc[subjects, 'Hypomania']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load and prepare data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Load cluster results.
results = read_csv(os.path.join(root_dir, 'results', '%s_%s_%s_results.csv' %(model,analysis,domain)))
results = results[(results.Contrast==contrast)&(results.FDR<fdr)].reset_index(drop=True)
results = results[results.Label == 'dlpfc_5-lh'] # Limit to the theta clusters.

## Load time series data.
npz = np.load(os.path.join(root_dir, 'afMSIT_%s_%s_%s_%s.npz' %(space, analysis, 'dlpfc_5-lh', 'theta')))
data = npz['data']
times = npz['times']

## Compute condition differences.
delta = np.zeros(subjects.shape[0])
mask = (times >= results.Tmin.min()) & (times <= results.Tmax.min())    

for m, subject in enumerate(subjects):
    i, = np.where((info['Subject']==subject)&(info[contrast]==0))
    j, = np.where((info['Subject']==subject)&(info[contrast]==1))
    delta[m] += (data[j][:,mask].mean(axis=0) - data[i][:,mask].mean(axis=0)).mean()

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Perform correlations.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Compute true correlations.
true_scores = spearmanr(mania, delta)

## Perform bootstrapping
n_subj = subjects.shape[0]
bootstraps = np.zeros(n_straps)

for m in np.arange(n_straps):

    ix = np.random.choice(np.arange(n_subj), n_subj, replace=True)
    bootstraps[m], _ = spearmanr(mania[ix], delta[ix])

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Plot Scatterplot.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

fig = plt.figure(figsize=(12,6))
ax = plt.subplot2grid((1,2),(0,0))
color = '#2b8cbe'

## Iteratively add points.
for x,y,subj in zip(mania,delta,mania.index):

    ax.scatter(x,y,s=80,color=color)
    ax.text(x+0.01,y+0.05,subj)

## Add trend line.
x1, x2 = ax.get_xlim()
y1, y2 = ax.get_ylim()

z = np.polyfit(mania, delta, 1)
p = np.poly1d(z)
ax.plot(np.linspace(x1,x2,1e3), p(np.linspace(x1,x2,1e3)), linewidth=2, color='k', alpha=0.8)
ax.set_xlim(x1,x2)
ax.set_ylim(y1,y2)

## Add flourishes.
ax.set_xlabel('Hypomania', fontsize=18)
ax.set_ylabel('Theta Power [On - Off]', fontsize=18)
ax.set_title('DLPFC-5 LH', fontsize=24)
ax.text(x1+0.01, y2*0.99, 'r = %0.3f\np = %0.3f' %(true_scores[0],true_scores[1]), 
        fontsize=18, va='top')

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Plot Histogram.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

ax = plt.subplot2grid((1,2),(0,1))

## Plot histogram.
x = bootstraps[~np.isnan(bootstraps)]
ax.hist(x, bins=40, color=color)

## Add true score.
y1, y2 = ax.get_ylim()
ax.vlines(true_scores[0], y1, y2, linestyle='--', linewidth=2)
ax.text(true_scores[0], y2*0.98, 'r = %0.3f' %true_scores[0], 
        ha='right', va='top', rotation='vertical', weight='heavy')

## Add flourishes.
ax.set_xlabel('Spearman R', fontsize=18) 
ax.set_ylabel('Frequency', fontsize=18)
ax.set_title('%s Bootstraps' %n_straps, fontsize=20)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Save.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#        

plt.tight_layout()
plt.show()

## Intertrial Interval Spectra Comparison
### Make ITI epochs
Each trial is 1.93s. On average 5.6s between each trial. Minimum time between trials is 3.0s between trials. We take 2s windows of time, centered at the time between trials, using the baseline for the proceeding trial.

In [ ]:
import os
import numpy as np
from mne import compute_covariance, Epochs, EpochsArray, find_events, read_proj, pick_types, set_log_level
from mne.io import Raw
from pandas import read_csv
set_log_level(verbose=False)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Subject level parameters.
subjects = ['BRTU', 'CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
task = 'msit'

## Filtering parameters.
l_freq = 0.5
h_freq = 50
l_trans_bandwidth = l_freq / 2.
h_trans_bandwidth = 1.0
filter_length = '20s'
n_jobs = 3

## Epoching parameters.
event_id = dict( FN=1, FI=2, NN=3, NI=4 )      # Alik's convention, isn't he smart!?
tmin = -1.25
tmax = 1.25
baseline = (-0.5, -0.1)
reject   = dict(eeg=150e-6)
flat     = dict(eeg=5e-7)
detrend = None
decim = 1

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load behavior.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT'
data_file = os.path.join( root_dir, 'behavior', 'afMSIT_group_data.csv' )
df = read_csv(data_file)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

for subj in subjects:
    
    print 'Loading data for %s.' %subj

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    # Define paths.
    root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT'
    raw_file  = os.path.join( root_dir, 'raw', '%s_%s_raw.fif' %(subj,task) )
    proj_file = os.path.join( root_dir, 'raw', '%s_%s-proj.fif' %(subj,task) )
    
    # Load data.
    raw = Raw(raw_file,add_eeg_ref=False,preload=True,verbose=False)
    proj = read_proj(proj_file)
    
    ## Add projections.
    proj = [p for p in proj if 'ref' not in p['desc']]
    raw.add_proj(proj, remove_existing=True)
    raw.set_eeg_reference()
    raw.apply_proj()
    
    ## Reduce dataframe to subject.
    data = df[df.Subject==subj]
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Make events.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    trial_onsets  = find_events(raw, stim_channel='Trig1', output='onset', min_duration=0.25, verbose=False)
    trial_offsets = find_events(raw, stim_channel='Trig2', output='offset', min_duration=0.25, verbose=False)
    
    ## Identify time at halfway point between end of trial(n-1) and start of trial(n).
    iti_lengths = raw.times[trial_onsets[1:,0]] - raw.times[trial_offsets[:-1,0]]
    iti_centers = raw.times[trial_offsets[:-1,0]] + iti_lengths / 2.

    ## Prepend first ITI center as 1.5 prior to first onset (corresponding to a 3s "ITI" before first trial).
    first_onset = raw.times[trial_onsets[0,0]]
    iti_centers = np.insert(iti_centers,0,first_onset - 1.5)

    ## Modify first event of DBS-on so that center of ITI prior to first onset occurs 1.5s before onset.
    ix = np.where( (data.DBS==1) & (data.Trial==1) )[0][0]
    first_onset = raw.times[trial_onsets[ix,0]]
    iti_centers[ix] = first_onset - 1.5

    ## Make events.
    events = np.zeros_like(trial_offsets)
    events.T[0] += raw.time_as_index(iti_centers)
    
    # Update event identifiers.
    n = 1
    for dbs in [0,1]:
        for cond in [0,1]:
            ix, = np.where((data.DBS==dbs)&(data.Interference==cond))
            events[ix,-1] = n
            n+=1
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Filter
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    Fs = raw.info['sfreq']
    raw.filter(l_freq = l_freq, h_freq = h_freq, filter_length=filter_length, n_jobs=n_jobs,
               l_trans_bandwidth=l_trans_bandwidth, h_trans_bandwidth=h_trans_bandwidth)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Make epochs.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    # Make initial ITI epochs.
    picks = pick_types(raw.info, meg=False, eeg=True, exclude='bads')
    epochs = Epochs(raw, events, event_id=event_id, tmin=tmin, tmax=tmax, 
                    baseline=None, picks=picks, reject=None, flat=None, proj=True, 
                    detrend=detrend, decim=decim)

    # Make baseline periods.
    bl = Epochs(raw, trial_onsets, tmin=baseline[0], tmax=baseline[1], 
                baseline=None, picks=picks, reject=None, flat=None, proj=True, 
                detrend=detrend, decim=decim)
    
    # Baseline correction.
    arr = epochs.get_data()
    bl = bl.get_data().mean(axis=-1)
    arr = (arr.T - bl.T).T
    
    # Finalize ITI epochs.
    epochs = EpochsArray(arr, epochs.info, events=epochs.events, tmin=epochs.tmin, 
                         baseline=None, reject=reject, flat=flat, 
                         proj=True, verbose=False)
    print epochs
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Save data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    epochs.save(os.path.join(root_dir,'ave','%s_%s_%s_iti-epo.fif' %(subj,task,h_freq)))
    
    print '\n#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#\n'
    
print 'Done.'

### Make source localized ITI epochs

In [ ]:
import os
import numpy as np
import pylab as plt
from mne import read_epochs, read_label, read_source_spaces, set_log_level
from mne.minimum_norm import apply_inverse_epochs, read_inverse_operator
from scipy.io import loadmat
set_log_level(verbose=False)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Subject level parameters.
subjects = ['BRTU','CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
task = 'msit'
analysis = 'iti_bl'
parc = 'april2016'
fmax = 50

## Source localization parameters.
method = 'dSPM'
snr = 1.0  
lambda2 = 1.0 / snr ** 2
pick_ori = 'normal'

## Labels
rois = ['dlpfc_5-lh']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Iteratively load and prepare data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

root_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/afMSIT'
fs_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/freesurfs'

## Prepare fsaverage source space.
src = read_source_spaces(os.path.join(fs_dir,'fscopy','bem','fscopy-oct-6p-src.fif'))
vertices_to = [src[n]['vertno'] for n in xrange(2)]
labels = [read_label(os.path.join(fs_dir,'fscopy','label','april2016','%s.label' %roi), subject='fsaverage')
          for roi in rois]

for subject in subjects:

    print 'Performing source localization: %s' %subject

    ## Load in epochs.
    epochs = read_epochs(os.path.join(root_dir,'ave','%s_%s_%s_%s-epo.fif' %(subject,task,fmax,analysis)))
    times = epochs.times
    
    ## Load in secondary files.
    inv = read_inverse_operator(os.path.join(root_dir,'cov','%s_%s_%s-inv.fif' %(subject,task,fmax)))
    morph_mat = loadmat(os.path.join(root_dir, 'morph_maps', '%s-fsaverage_morph.mat' %subject))['morph_mat']

    ## Make generator object.
    G = apply_inverse_epochs(epochs, inv, method=method, lambda2=lambda2, pick_ori=pick_ori, return_generator=True)
    del inv

    ## Iteratively compute and store label timecourse. 
    ltcs = []
    for g in G:
        g = g.morph_precomputed('fsaverage', vertices_to=vertices_to, morph_mat=morph_mat)
        ltcs.append( g.extract_label_time_course(labels, src, mode='pca_flip') )
    ltcs = np.array(ltcs)
    
    ## Save.
    f = os.path.join(root_dir,'source','stcs','%s_%s_%s_%s_%s_epochs' %(subject,task,analysis,method,fmax))
    np.savez_compressed(f, ltcs=ltcs, times=times, labels=np.array([l.name for l in labels]), conds=epochs.events.T[-1])
    del ltcs
    
print 'Done.'

### Make power baseline for ITI spectral epochs

In [ ]:
import os
import numpy as np
from mne.filter import low_pass_filter
from mne.time_frequency import single_trial_power
from pandas import read_csv

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Data parameters.
subjects = ['BRTU','CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
analysis = 'stim'
method = 'dSPM'
h_freq = 50

## Label parameters.
rois = [ 'dlpfc_5-lh']

## Time-frequency parameters.
freqs = np.logspace( np.log10(2), np.log10(50), num=25)
n_cycles = 3
baseline = (-0.5, -0.1)
Fs = 1450.
decim = 14
n_jobs = 3

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/source'
df = read_csv(os.path.join(root_dir, 'afMSIT_source_info.csv'))

for roi in rois:

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Iteratively load and merge.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ltcs = []
    for subject in subjects:

        ## Load NPZ.
        npz = np.load(os.path.join(root_dir, 'stcs', '%s_msit_%s_%s_%s_epochs.npz' %(subject,analysis,method,h_freq)))

        ## Get label index and extract.
        ix = npz['labels'].tolist().index(roi)
        arr = npz['ltcs'][:,ix,:]

        ## Append.
        ltcs.append(arr)

    ## Concatenate.
    ltcs = np.concatenate(ltcs, axis=0)
    times = npz['times']

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Phase-lock removal.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    for subject in df.Subject.unique():
        for dbs in [0, 1]:
            for cond in [0, 1]:
                ix, = np.where((df.Subject==subject)&(df.DBS==dbs)&(df.Interference==cond))
                ltcs[ix] -= ltcs[ix].mean(axis=0)

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Compute power.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    print 'Computing single trial power: %s %s.' %(roi, analysis)
    ltcs = np.expand_dims(ltcs,1)
    power = single_trial_power(ltcs, sfreq=Fs, frequencies=freqs, n_cycles=n_cycles, 
                                baseline=None, n_jobs=n_jobs, verbose=False)
    power = power.squeeze()

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Within-trial normalization.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    power = (power.T / np.median(power, axis=-1).T).T # median, not mean

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Prepare baseline normalization (within subject, DBS).
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Compute baseline normalization.
    if analysis == 'stim':

        ## Make time mask.
        mask = (times >= baseline[0]) & (times <= baseline[1])

        ## Iteratively compute over subjects.
        blnorm = []
        for subject in df.Subject.unique():

            ## Iteratively compute over DBS conditions..
            sbl = []
            for dbs in [0,1]:

                ix, = np.where((df.Subject==subject)&(df.DBS==dbs))
                sbl.append( np.apply_over_axes(np.median, power[ix][:,:,mask], axes=[0,2]).squeeze() )

            blnorm.append(sbl)

        ## Merge.
        blnorm = np.array(blnorm)
      
        ## Save.
        np.save(os.path.join(root_dir, 'afMSIT_source_iti_%s_bl' %roi), blnorm)
        
print 'Done.'

### Compute power of source timecourses.

In [ ]:
import os
import numpy as np
from mne.filter import low_pass_filter
from mne.time_frequency import single_trial_power
from pandas import DataFrame

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Data parameters.
subjects = ['BRTU','CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
method = 'dSPM'
h_freq = 50

## Label parameters.
rois = ['dlpfc_5-lh']

## Time-frequency parameters.
freqs = np.logspace( np.log10(2), np.log10(50), num=25)
n_cycles = 3
Fs = 1450.
decim = 14
n_jobs = 3

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/source'

for roi in rois:

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Iteratively load and merge.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ltcs = []
    df = dict(Condition = [], Subject = [])
    
    for subject in subjects:

        ## Load NPZ.
        npz = np.load(os.path.join(root_dir, 'stcs', '%s_msit_iti_%s_%s_epochs.npz' %(subject,method,h_freq)))

        ## Get label index and extract.
        ix = npz['labels'].tolist().index(roi)
        arr = npz['ltcs'][:,ix,:]

        ## Phase-lock removal.
        for cond in [1,2,3,4]:
            ix, = np.where(npz['conds']==cond)
            arr[ix] -= arr[ix].mean(axis=0)
        
        ## Append.
        ltcs.append(arr)
        df['Condition'] += npz['conds'].tolist()
        df['Subject'] += [subject] * len(npz['conds'])

    ## Concatenate.
    ltcs = np.concatenate(ltcs, axis=0)
    times = npz['times']
    df = DataFrame(df)
    df['DBS'] = (df.Condition > 2).astype(int)

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Compute power.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    print 'Computing single trial power: %s.' %roi
    ltcs = np.expand_dims(ltcs,1)
    power = single_trial_power(ltcs, sfreq=Fs, frequencies=freqs, n_cycles=n_cycles, 
                                baseline=None, n_jobs=n_jobs, verbose=False)
    power = power.squeeze()

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Within-trial normalization.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    power = (power.T / np.median(power, axis=-1).T).T # median, not mean

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Apply baseline normalization (within subject, DBS).
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Load baseline for for normalization.
    blnorm = np.load(os.path.join(root_dir, 'afMSIT_source_iti_%s_bl.npy' %roi))
    
    ## Setup index vectors.
    n_trials, n_freqs, n_times = power.shape        
    _, subj_ix = np.unique(df.Subject, return_inverse=True)
    dbs_ix = df.DBS.as_matrix()
    trial_ix = np.arange(n_trials) 

    ## Main loop.
    for i,j,k in zip(trial_ix,subj_ix,dbs_ix):

        for m in xrange(n_times):

            power[i,:,m] /= blnorm[j,k,:]

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Final preprocessing steps.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Crop times.
    mask = (times >= -1) & (times <= 1)
    power = power[:, :, mask]
    times = times[mask]

    ## Decimate
    power = power[:,:,::decim]
    times = times[::decim]
    
    ## Save.
    f = os.path.join(root_dir, 'afMSIT_source_iti_%s_spectra' %roi)
    np.savez_compressed(f, power=power, times=times, freqs=freqs, n_cycles=n_cycles, 
                        subject=df.Subject.as_matrix(), conds=df.Condition.as_matrix())

    del power

print 'Done.'

## Resting State Spectral Comparisons
### Projections: EOG

In [ ]:
import os
from mne import write_proj
from mne.preprocessing import compute_proj_eog
from mne.io import Raw

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Setup
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## File params.
root_dir = '/space/sophia/2/users/EMOTE-DBS/resting/raw'
subjects = ['S2']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main Loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
raw_files = os.listdir(root_dir)
raw_files = [f for f in raw_files if f.split('_')[0] in subjects and f.endswith('raw.fif')]

for raw_file in raw_files:    
    
    ## Load files.
    raw_file = os.path.join(root_dir, raw_file)
    raw = Raw(raw_file, preload=True, verbose=False, add_eeg_ref=False)
    raw.add_eeg_average_proj()
    
    proj, _ = compute_proj_eog(raw, n_eeg = 4, average=True, filter_length='20s',
                               reject=dict(eeg=5e-4), flat=dict(eeg=5e-8),  ch_name='EOG', n_jobs=3)
    write_proj(raw_file.replace('_raw.fif', '_eog-proj.fif'), proj)

### Projections: ECG

In [ ]:
import os
from mne import read_proj, write_proj
from mne.preprocessing import compute_proj_ecg
from mne.io import Raw

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Setup
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
## File params.
root_dir = '/space/sophia/2/users/EMOTE-DBS/resting/raw'
subjects = ['CHDR']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main Loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
raw_files = os.listdir(root_dir)
raw_files = [f for f in raw_files if f.split('_')[0] in subjects and f.endswith('raw.fif')]

for raw_file in raw_files:    
    
    ## Load files.
    raw_file = os.path.join(root_dir, raw_file)
    raw = Raw(raw_file, preload=True, verbose=False, add_eeg_ref=False)
    raw.add_eeg_average_proj()
    
    ## Make ECG proj. Save.
    proj, _ = compute_proj_ecg(raw, n_eeg = 4, h_freq = 35., average=True, filter_length='20s',
                                reject=dict(eeg=5e-4), flat=dict(eeg=5e-8), ch_name='P9', n_jobs=3)
    write_proj(raw_file.replace('_raw.fif', '_ecg-proj.fif'), proj)

### Compute PSD

In [ ]:
import os
import numpy as np
import pylab as plt
from mne import pick_channels, read_proj, set_log_level
from mne.io import Raw
from mne.time_frequency import psd_multitaper
set_log_level(verbose=False)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define Parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
subjects = ['BRTU', 'CHDR', 'JADE', 'S2']
conds = ['resting_dbsoff_eo', 'resting_dbson_eo']

## Filtering parameters.
fmin, fmax = 0.5, 30
tmin, tmax = 10, 40
chs = ['FCZ']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
root_dir = '/space/sophia/2/users/EMOTE-DBS/resting/raw'

psds = []
for subject in subjects:
    
    psds.append([])
    
    for cond in conds:
        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Prepare data. 
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
        ## Load data.
        raw = os.path.join(root_dir, '%s_%s_raw.fif' %(subject,cond))
        raw = Raw(raw, preload=True, add_eeg_ref=False, verbose=False)
        
        ## Load/apply projection(s).
        proj = os.path.join(root_dir, '%s_%s-proj.fif' %(subject,cond))
        
        if os.path.isfile(proj): 
            proj = read_proj(proj)
            raw.add_proj(proj)
        else:
            raw.set_eeg_reference()
        
        raw.apply_proj()
        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Compute PSD.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
        picks = pick_channels(raw.ch_names, chs)
        psd, freqs = psd_multitaper(raw, fmin=fmin, fmax=fmax, tmin=10, tmax=40, picks=picks, n_jobs=3)
        psds[-1].append(psd)
    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Merge and plot.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Merge.
psds = np.array(psds).squeeze()

In [ ]:
## Normalize.
psd = psds.mean(axis=0)
auc = psd.sum(axis=-1)
psd = (psd.T / auc).T

## Plot.
fig, ax = plt.subplots(1,1,figsize=(10,5))
for arr, label, color in zip(psd, ['DBSoff','DBSon'], ['#0571b0','#ca0020']):
    ax.plot(freqs, arr, linewidth=4, label=label, color=color, alpha=0.7)
ax.legend(loc=1, frameon=False, fontsize=18)

plt.tight_layout()
# plt.close('all')
plt.show()